# Graph Classification

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

### MUTAG

In [8]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 150
data_path = "/data/XXX/Pooling/1"
dataset_sparse = TUDataset(root=data_path, name="MUTAG", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_ASA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_ASA, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = ASAPooling(hidden_channels, ratio=0.9)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = ASAPooling(hidden_channels, ratio=0.9)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool1(x, edge_index, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Early stopping at epoch 151 for seed 43
Average Time: 114.61 seconds
Var Time: 768.89 seconds
Average Memory: 80.00 MB
Average Best Val Acc: 0.6667
Std Best Test Acc: 0.0709
Average Test Acc: 0.6667


### DD

In [2]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/XXX/Pooling/"
dataset_sparse = TUDataset(root=data_path, name="DD", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_ASA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_ASA, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = ASAPooling(hidden_channels, ratio=0.7)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = ASAPooling(hidden_channels, ratio=0.7)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool1(x, edge_index, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6919, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 002, Loss: 0.6912, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 003, Loss: 0.6908, Val Acc: 0.5495, Test Acc: 0.5586


### IMDB-MULTI

In [3]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/XXX/Pooling/"
dataset_sparse = TUDataset(root=data_path, name="IMDB-MULTI", transform=T.Compose([T.OneHotDegree(88)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_ASA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_ASA, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = ASAPooling(hidden_channels, ratio=0.9)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = ASAPooling(hidden_channels, ratio=0.9)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool1(x, edge_index, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 1.1033, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 002, Loss: 1.1026, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 003, Loss: 1.1021, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 004, Loss: 1.1016, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 005, Loss: 1.1009, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 006, Loss: 1.1004, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 007, Loss: 1.0997, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 008, Loss: 1.0990, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 009, Loss: 1.0984, Val Acc: 0.2711, Test Acc: 0.3600
Seed: 42, Epoch: 010, Loss: 1.0972, Val Acc: 0.2711, Test Acc: 0.3600
Seed: 42, Epoch: 011, Loss: 1.0968, Val Acc: 0.2711, Test Acc: 0.3600
Seed: 42, Epoch: 012, Loss: 1.0951, Val Acc: 0.2711, Test Acc: 0.3600
Seed: 42, Epoch: 013, Loss: 1.0939, Val Acc: 0.2756, Test Acc: 0.3556
Seed: 42, Epoch: 014, Loss: 1.0932, Val Acc: 0.2800, Test Acc: 0.3689
Seed: 42, Epoch: 015

### COLLAB

In [2]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
data_path = "/data1/Pooling/"
dataset_sparse = TUDataset(root=data_path, name="COLLAB", transform=T.Compose([T.OneHotDegree(491)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class HierarchicalGCN_ASA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_ASA, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = ASAPooling(hidden_channels, ratio=0.9)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = ASAPooling(hidden_channels, ratio=0.9)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool1(x, edge_index, batch=batch)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, batch=batch)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)
num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_sparse = dataset_sparse.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
    model = HierarchicalGCN_ASA(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

# Graph Regression

### QM7

In [1]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --asa_ratio=0.1 --pooling='ASAP'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --asa_ratio=0.3 --pooling='ASAP'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --asa_ratio=0.5 --pooling='ASAP'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --asa_ratio=0.7 --pooling='ASAP'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --run_times=5 --patience=150 --epochs=500 --asa_ratio=0.9 --pooling='ASAP'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Generating dataset...
Packaging molecules, finish 100.0%

Current dataset: qm7, include 6832 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/500MAE=1549.9890 MAE=1548.3679 MAE=1544.8348 MAE=1542.1368 MAE=1540.3928 MAE=1536.1909 MAE=1533.2390 MAE=1526.7466 MAE=1523.4377 Epoch: 10/500MAE=1518.5776 MAE=1513.7883 MAE=1512.1201 MAE=1506.9475 MAE=1494.6902 MAE=1487.8940 MAE=1475.9880 MAE=1472.7158 MAE=1464.3601 MAE=1459.8149 Epoch: 20/500MAE=1453.1532 MAE=1440.7437 MAE=1435.9729 MAE=1433.3440 MAE=1416.6311 MAE=1395.0476 MAE=1384.8713 MAE=1373.6216 MAE=1366.0151 MAE=1349.3988 Epoch: 30/500MAE=1345.4993 MAE=1326.1885 MAE=1317.5447 MAE=1301.8408 MAE=1287.2114 MAE=1278.6711 MAE=1258.0403 MAE=1246.9518 MAE=1228.2086 MAE=1183.9573 Epoch: 40/500MAE=1184.0028 MAE=1190.0552 MAE=1167.0938 MAE=1140.5122 MAE=1130.6899 MAE=1098.1836 MAE=1108.6740 MAE=1081.7876 MAE=1036.7192 MAE=1010.5583 Epoch: 50/500MAE=993.3591 MAE=987.4745 MAE=949.94

### QM8

In [2]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --asa_ratio=0.1 --pooling='ASAP'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --asa_ratio=0.3 --pooling='ASAP'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --asa_ratio=0.5 --pooling='ASAP'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --asa_ratio=0.7 --pooling='ASAP'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --run_times=5 --patience=10 --epochs=150 --asa_ratio=0.9 --pooling='ASAP'

++++++++++++++++++++++0.1++++++++++++++++++++++++
Generating dataset...
Packaging molecules, finish 100.0%

Current dataset: qm8, include 21783 molecules and 12 regression tasks

Splitting, finish 1/1  
Epoch: 1/150MAE=0.1424 MAE=0.1349 MAE=0.1242 MAE=0.1136 MAE=0.1028 MAE=0.0883 MAE=0.0732 MAE=0.0555 MAE=0.0437 Epoch: 10/150MAE=0.0439 MAE=0.0347 MAE=0.0385 MAE=0.0390 MAE=0.0330 MAE=0.0350 MAE=0.0394 MAE=0.0441 MAE=0.0536 MAE=0.0434 Epoch: 20/150MAE=0.0387 MAE=0.0432 MAE=0.0350 MAE=0.0319 MAE=0.0313 MAE=0.0307 MAE=0.0308 MAE=0.0297 MAE=0.0302 MAE=0.0293 Epoch: 30/150MAE=0.0289 MAE=0.0315 MAE=0.0318 MAE=0.0329 MAE=0.0379 MAE=0.0358 MAE=0.0322 MAE=0.0291 MAE=0.0282 MAE=0.0267 Epoch: 40/150MAE=0.0266 MAE=0.0269 MAE=0.0277 MAE=0.0306 MAE=0.0284 MAE=0.0272 MAE=0.0271 MAE=0.0270 MAE=0.0270 MAE=0.0265 Epoch: 50/150MAE=0.0267 MAE=0.0270 MAE=0.0266 MAE=0.0261 MAE=0.0260 MAE=0.0260 MAE=0.0259 MAE=0.0259 MAE=0.0259 MAE=0.0259 Epoch: 60/150MAE=0.0260 MAE=0.0264 MAE=0.0261 MAE=0.0260 MAE=0.0260 MAE

### BACE

In [ ]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 2 --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.1 --pooling='ASAP'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 2 --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.3 --pooling='ASAP'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 2 --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.5 --pooling='ASAP'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 2 --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.7 --pooling='ASAP'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num 2 --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.9 --pooling='ASAP'

### ESOL

In [ ]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.1 --pooling='ASAP'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.3 --pooling='ASAP'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.5 --pooling='ASAP'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.7 --pooling='ASAP'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.9 --pooling='ASAP'

### Freesolv

In [ ]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.1 --pooling='ASAP'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.3 --pooling='ASAP'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.5 --pooling='ASAP'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.7 --pooling='ASAP'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.9 --pooling='ASAP'

### Lipophilicity

In [ ]:
print("++++++++++++++++++++++0.1++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.1 --pooling='ASAP'
print("++++++++++++++++++++++0.3++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.3 --pooling='ASAP'
print("++++++++++++++++++++++0.5++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.5 --pooling='ASAP'
print("++++++++++++++++++++++0.7++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.7 --pooling='ASAP'
print("++++++++++++++++++++++0.9++++++++++++++++++++++++")
!python /data1/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --asa_ratio=0.9 --pooling='ASAP'